In [192]:
def get_links(link, tag, class_name):
    import urllib.request as urllib
    from bs4 import BeautifulSoup
    html = urllib.urlopen(link)
    soup = BeautifulSoup(html.read())
    each_recipe = soup.body.find_all(tag, class_= class_name)
    links = []
    for recipe in each_recipe:
        links.append(recipe.find('a').get('href').strip())
    return links;

In [193]:
def replace_frac(string):
    from unicodedata import numeric
    import re as re
    string = string.replace('-', ' ')
    if string.split()[0].isalpha():
        string_to_replace = re.search(r'\d+', string).group()
    else:
        string_to_replace = string.split()[0]
    string_replacement = ''
    if string_to_replace.find('/') != -1:
        string_replacement = str(float(string_to_replace[string_to_replace.find('/')-1]) / float(string_to_replace[string_to_replace.find('/')+1]))
    elif len(string_to_replace) == 1:
        string_replacement = numeric(string_to_replace)
    elif string_to_replace[-1].isdigit():
        string_replacement = float(string_to_replace)
    else:
        string_replacement = float(string_to_replace[:-1]) + numeric(string_to_replace[-1])
    string = string.replace(string_to_replace, str(string_replacement))
    return string;

In [203]:
def get_title(html, tag, class_name):
    return html.body.find(tag, class_= class_name).getText();

def get_time(html, tag, class_name):
    import re as re
    time = html.body.find(tag, class_=class_name).getText()
    if time.find('/') != -1:
        time = (int(time[0]) + float(time.find('/') - 1) / (float(time.find('/') + 1))) * 60
    elif time.find('hr') != -1 and time.find('min') != -1:
        time = (int(re.search(r'\d+', time[time.find('hr')-2]).group()) * 60) + int(re.search(r'\d+', time[time.find('min')-2]).group())
    elif time.find('hr') != -1:
        time = int(re.search(r'\d+', time).group()) * 60
    else:
        time = int(re.search(r'\d+', time).group())
    return time;
    
def get_amount(html, tag, class_name, tag_2, class_2):
    import re as re
    amount = html.body.find(tag, class_name)
    if(amount is not None) :
        amount = amount.find(tag_2, class_=class_2).getText()
        amount = replace_frac(amount)
        if amount.isdigit():
            amount = float(re.search(r'\d+', amount).group())
        else:
            amount = float(re.search(r'\d+\.\d*', amount).group())
    else:
        amount = 1
    return amount;
    
def get_instructions(html, tag, class_name, tag_2, class_2):
    instruction = ''
    for step in html.body.find(tag, class_=class_name).find_all(tag_2, class_=class_2) :
        instruction = instruction + step.getText().replace('\n', '')
        instruction = instruction.strip('  ')
    return instruction;

def get_image(html, tag, class_name):
    image = html.body.find(tag, class_=class_name).get('src')
    image = "https:" + image
    return image;

def get_description(html, tag, class_name):
    description = html.body.find(tag, class_=class_name)
    if description is not None:
        return description.getText().replace("\n", "")
    else:
        return ""

In [221]:
def get_ingredients(html):
    s = []
    for input in html.find_all('input'):
        if input['type'] in ('checkbox'):
            value = ''
            if input.has_attr('checked'):
                if input.has_attr('value'):
                    value = input['value']
                    is_digit = False;
                    for character in value:
                        if(character.isdigit()):
                            is_digit = True
                    if(is_digit):
                        s.append(replace_frac(value))
                    elif(value != 'Deselect All'):
                        s.append(value)
    return s;

def get_ingredient_name(ingredients):
    import re as re
    unit_list = ['ounce', 'zest', 'pound', 'gram', 'tablespoon', 'teaspoon', 'cup', 'large', 'medium', 'small', 'clove', 'sprig', 'stalk', 'slice', 'quart', 'whole']
    name = []
    try:
        for ingredient in ingredients:
            if(len(ingredient.split()) > 1):
                if ingredient.split()[1].find('to') != -1 and len(ingredient.split()):
                    ingredient = ingredient.split(' ', 4)[4].replace("\xa0", " ").replace("\n", " ").replace("\t", " ").replace('of', '').strip('1234567890.-/ ')
                elif ingredient.split()[1].rstrip('s') not in unit_list:
                    ingredient = ingredient.split(' ', 1)[1].replace("\xa0", " ").replace("\n", " ").replace("\t", " ").replace('of', '').strip('1234567890.-/ ')
                else:
                    ingredient = ingredient.split(' ', 2)[2].replace("\xa0", " ").replace("\n", " ").replace("\t", " ").replace('of', '').strip('1234567890.-/ ')
                if ingredient.find(',') != -1:
                    ingredient = ingredient[:ingredient.find(',')]
                for unit in unit_list:
                    if ingredient.find(unit) != -1:
                        ingredient = ingredient.replace(unit, '')
            name.append(ingredient.strip())
    except IndexError:
        pass
    return name;

def get_ingredient_amount(ingredients):
    import re as re
    amount = []
    for ingredient in ingredients:
        if re.search(r'\d+\.\d*', ingredient) is not None:
            amount.append(float(re.search(r'\d+\.\d*', ingredient).group()))
        else:
            amount.append(1.0)
    return amount;

def get_ingredient_unit(ingredients):
    import re as re
    unit_list = ['ounce', 'zest', 'pound', 'gram', 'tablespoon', 'teaspoon', 'cup', 'large', 'medium', 'small', 'clove', 'sprig', 'stalk', 'slice', 'quart', 'whole', 'to taste']
    unit = []
    for ingredient in ingredients:
        if len(ingredient.split()) > 1:
            if re.search(r'\d+\.\d*', ingredient) is None:
                ingredient = 'to taste'
            elif(ingredient.split()[1].find('to') != -1):
                ingredient = ingredient.split(' ', 4)[3]
            elif ingredient.find('-ounce') != -1:
                ingredient = 'ounce'
            elif ingredient.find('zest') != -1:
                ingredient = 'zest'
            else:
                ingredient = ingredient.split()[1]
            ingredient = ingredient.rstrip('s');
            if ingredient not in unit_list:
                ingredient = 'whole'
        unit.append(ingredient)
    return unit;

def get_ingredient_opt(ingredients):
    import re as re
    opt = []
    for ingredient in ingredients:
        if ingredient.find("opt") != -1 or re.search(r'\d+\.\d*', ingredient) is None:
            opt.append(True)
        else:
            opt.append(False)
    return opt;



In [265]:
def find_appliances(instruction, ingredients):
    appliance = []
    appliances_dict = {"name": appliance}
    if(instruction.find("Dutch oven")!= -1 or instruction.find("pot") != -1):
        appliance.append('pot')
    if(instruction.find("blender")!= -1 or instruction.find("blend") != -1 or instruction.find('puree') != -1):
        appliance.append("blender")
    if(instruction.find("toaster")!= -1):
        appliance.append("toaster")
    if(instruction.find("stove")!= -1):
        appliance.append("stove")
    if(instruction.find("microwave")!= -1):
        appliance.append("microwave")
    if instruction.find("cut")!= -1 or instruction.find("chop") != -1 or instruction.find("knife") !=-1 or instruction.find('dice') !=-1 or instruction.find('mince') or ingredients.find("cut")!= -1 or ingredients.find("chop") != -1 or ingredients.find("knife") !=-1 or ingredients.find('dice') !=-1 or ingredients.find('mince'):
        appliance.append("knife")
    if(instruction.find("spatula")!= -1):
        appliance.append("spatula")
    if(instruction.find("whisk")!= -1):
        appliance.append("whisk")
    if(instruction.find("mixer")!= -1):
        appliance.append("mixer")
    if(instruction.find("saucepan")!= -1):
        appliance.append("saucepan")
    if(instruction.find("skillet") != -1):
        appliance.append("skillet")
    if(instruction.find('bake') != -1 or instruction.find('preheat') != -1 or instruction.find('oven') != -1):
        appliance.append('oven')
    if instruction.find('sheet pan') != -1 or instruction.find('baking sheet') != -1 or instruction.find('baking pan') != -1 or instruction.find('tray') != -1:
        appliance.append('sheet pan')
    if instruction.find('baking dish') != -1:
        appliance.append('baking dish')
    return appliances_dict;

In [267]:
# specific for the food network website of pantry recipes, would need to change link, tags, and class names
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
recipes_1 = pd.DataFrame(columns = ["title", "description", "image", "minutes", "servings", "instructions"])
ingredients_1 = pd.DataFrame(columns = ["name", "amount", "unit", "optional"])
ingredients_1['name'] = ingredients_1['name'].astype(object)
ingredients_1['amount'] = ingredients_1['amount'].astype(object)
ingredients_1['unit'] = ingredients_1['unit'].astype(object)
ingredients_1['optional'] = ingredients_1['optional'].astype(object)
appliances_1 = pd.DataFrame(columns = ['name'])
appliances_1['name'] = appliances_1['name'].astype(object)
for link in get_links("https://www.foodnetwork.com/recipes/photos/cooking-from-the-pantry", 'p', 'o-PhotoGalleryPromo__a-Cta'):  
    html_1 = urllib.urlopen('https:' + link)
    soup_1 = BeautifulSoup(html_1.read())
    ingredients = get_ingredients(soup_1)
    #define recipe and ingredient dictionary
    recipe_dict = {"title":get_title(soup_1, "span", "o-AssetTitle__a-HeadlineText"),
                   "description":get_description(soup_1, 'div', 'o-AssetDescription__a-Description'),
                    "image":get_image(soup_1, "img", "m-MediaBlock__a-Image a-Image"), 
                    "minutes":get_time(soup_1, "span", "o-RecipeInfo__a-Description m-RecipeInfo__a-Description--Total"), 
                    "servings":get_amount(soup_1, "ul", "o-RecipeInfo__m-Yield", "span", "o-RecipeInfo__a-Description"), 
                    "instructions":get_instructions(soup_1, 'div', 'o-Method__m-Body', 'li', 'o-Method__m-Step')}
    ingredients_dict = {"name": get_ingredient_name(ingredients), 
                        "amount": get_ingredient_amount(ingredients),
                        "unit": get_ingredient_unit(ingredients),
                        "optional":get_ingredient_opt(ingredients)}
    #append the dictionary to the dfs
    recipes_1 = recipes_1.append(recipe_dict, ignore_index=True)
    ingredients_1 = ingredients_1.append(ingredients_dict, ignore_index = True)
    appliances_1 = appliances_1.append(find_appliances(get_instructions(soup_1, 'div', 'o-Method__m-Body', 'li', 'o-Method__m-Step'), ingredients), ignore_index = True)

In [268]:
recipe_1appliances_1.to_csv('food_network_appliances_1.csv')

In [266]:
# specific for the food network website of pantry recipes, would need to change link, tags, and class names
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
recipes_2 = pd.DataFrame(columns = ["title", "description", "image", "minutes", "servings", "instructions"])
ingredients_2 = pd.DataFrame(columns = ["name", "amount", "unit", "optional"])
ingredients_2['name'] = ingredients_2['name'].astype(object)
ingredients_2['amount'] = ingredients_2['amount'].astype(object)
ingredients_2['unit'] = ingredients_2['unit'].astype(object)
ingredients_2['optional'] = ingredients_2['optional'].astype(object)
appliances_2 = pd.DataFrame(columns = ['name'])
appliances_2['name'] = appliances_2['name'].astype(object)
for link in get_links("https://www.foodnetwork.com/recipes/packages/comfort-foods/easy-comfort-food-recipes/easy-comfort-food-recipes", 'p', 'o-PhotoGalleryPromo__a-Cta'):  
    html_2 = urllib.urlopen('https:' + link)
    soup_2 = BeautifulSoup(html_2.read())
    ingredients2 = get_ingredients(soup_2)
    #define recipe and ingredient dictionary
    print(link)
    recipe_dict = {"title":get_title(soup_2, "span", "o-AssetTitle__a-HeadlineText"),
                   "description":get_description(soup_2, 'div', 'o-AssetDescription__a-Description'),
                    "image":get_image(soup_2, "img", "m-MediaBlock__a-Image a-Image"), 
                    "minutes":get_time(soup_2, "span", "o-RecipeInfo__a-Description m-RecipeInfo__a-Description--Total"), 
                    "servings":get_amount(soup_2, "ul", "o-RecipeInfo__m-Yield", "span", "o-RecipeInfo__a-Description"), 
                    "instructions":get_instructions(soup_2, 'div', 'o-Method__m-Body', 'li', 'o-Method__m-Step')}
    ingredients_dict = {"name": get_ingredient_name(ingredients2), 
                        "amount": get_ingredient_amount(ingredients2),
                        "unit": get_ingredient_unit(ingredients2),
                        "optional":get_ingredient_opt(ingredients2)}
    #append the dictionary to the dfs
    recipes_2 = recipes_2.append(recipe_dict, ignore_index=True)
    ingredients_2 = ingredients_2.append(ingredients_dict, ignore_index = True)
    appliances_2 = appliances_2.append(find_appliances(get_instructions(soup_2, 'div', 'o-Method__m-Body', 'li', 'o-Method__m-Step'), ingredients2), ignore_index = True)


//www.foodnetwork.com/recipes/food-network-kitchen/buffalo-cauliflower-baked-mac-and-cheese-3626905
//www.foodnetwork.com/recipes/food-network-kitchen/bacon-egg-and-cheese-breakfast-burgers-5484039
//www.foodnetwork.com/recipes/food-network-kitchen/creamy-spinach-and-artichoke-chicken-skillet-3885792
//www.foodnetwork.com/recipes/food-network-kitchen/oven-fried-ranch-chicken-recipe-2109282
//www.foodnetwork.com/recipes/ree-drummond/garlic-cheese-bread-sticks-2200150
//www.foodnetwork.com/recipes/ree-drummond/meatloaf-recipe-2042541
//www.foodnetwork.com/recipes/food-network-kitchen/scalloped-potatoes-recipe-2011508
//www.foodnetwork.com/recipes/food-network-kitchen/southern-fried-chicken-parmesan-5337285
//www.foodnetwork.com/recipes/food-network-kitchen/the-best-chicken-noodle-soup-7194859
//www.foodnetwork.com/recipes/food-network-kitchen/spinach-and-artichoke-stuffed-shells-4594233
//www.foodnetwork.com/recipes/food-network-kitchen/chicken-and-herb-dumplings-7442829
//www.foodnetwor

//www.foodnetwork.com/recipes/food-network-kitchen/eggplant-parmesan-hero-recipe-3361492
//www.foodnetwork.com/recipes/ree-drummond/lasagna-2111724
//www.foodnetwork.com/recipes/tyler-florence/the-ultimate-sloppy-joes-recipe-2013701
//www.foodnetwork.com/recipes/food-network-kitchen/chicken-alfredo-stuffed-shells-8022884
//www.foodnetwork.com/recipes/food-network-kitchen/instant-pot-creamy-ranch-chicken-pasta-8100706
//www.foodnetwork.com/recipes/food-network-kitchen/chicken-parmesan-lasagna-8849512


In [259]:
recipes_2.drop(index = [65, 66, 86, 93], axis = 0, inplace = True)
recipes_2.reset_index(drop = True, inplace = True)
recipes_2.to_csv("recipes_2.csv")

In [269]:
appliances_2.drop(index = [65, 66, 86, 93], axis = 0, inplace = True)
appliances_2.reset_index(drop = True, inplace = True)
appliances_2.to_csv("appliances_2.csv")

In [255]:
ingredients_2.drop(index = [65, 66, 86, 93], axis = 0, inplace = True)
ingredients_2.reset_index(drop = True, inplace = True)
ingredients_2.to_csv("ingredients_2.csv")